In [1]:
# Install lightgbm if not already installed
%pip install lightgbm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Import the libraries

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
print("added")

added


### Loading of the train data

In [3]:
# Load the dataset
print("Loading data...")
try:
    train_orders = pd.read_csv('Train/orders.csv', low_memory=False)
    train_customers = pd.read_csv('Train/train_customers.csv')
    train_locations = pd.read_csv('Train/train_locations.csv')
    vendors = pd.read_csv('Train/vendors.csv')
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure all CSV files are in the same directory as the script.")
    exit()

print("Data loaded successfully.")

Loading data...
Data loaded successfully.
Data loaded successfully.


In [4]:
print("Loading data...")

try:
    # --- Load all source files ---
    train_orders = pd.read_csv('Train/orders.csv')
    train_customers = pd.read_csv('Train/train_customers.csv')
    train_locations = pd.read_csv('Train/train_locations.csv')
    vendors = pd.read_csv('Train/vendors.csv')

except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please ensure all CSV files are in the correct 'Train/' subdirectory.")
    exit()

print("Preparing and merging data...")

# --- Rename columns BEFORE merging to avoid confusion ('_x', '_y') ---
vendors.rename(columns={
    'latitude': 'vendor_lat',
    'longitude': 'vendor_lon',
    'status': 'vendor_status',
    'rating': 'vendor_rating'
}, inplace=True)

train_locations.rename(columns={
    'latitude': 'customer_lat',
    'longitude': 'customer_lon'
}, inplace=True)

# --- Merge all training data sources ---
# Start with orders and add details about the customer, vendor, and location
train_merged = train_orders.merge(train_customers, on='customer_id', how='left')
train_merged = train_merged.merge(vendors, left_on='vendor_id', right_on='id', how='left')
train_merged = train_merged.merge(
    train_locations,
    on=['customer_id'],  # Only merge on customer_id
    how='left'
)

# Debug: print columns to check for missing/misnamed columns
print("\nColumns in train_merged:")
print(train_merged.columns.tolist())

# --- Define the specific columns required for training a model ---
# These features are known at the time of prediction and avoid data leakage
required_columns = [
    # --- IDs (for context, not as model features) ---
    'customer_id',
    'vendor_id',
    # 'LOCATION_NUMBER',  # Remove if not present

    # --- Customer Features ---
    'gender',
    'dob',                         # To calculate customer age
    'status',                      # Customer account status
    'created_at_x',                # To calculate customer tenure (from customers table)

    # --- Vendor Features ---
    'vendor_category_en',
    'delivery_charge',
    'serving_distance',
    'is_open',
    'prepration_time',             # Vendor's average preparation time
    'commission',
    'discount_percentage',
    'vendor_status',               # Vendor's account status
    'rank',
    # 'vendor_rating',               # Vendor's overall historical rating (removed)
    'vendor_tag_name',             # Descriptive tags like 'Healthy', 'Pizza'

    # --- Location & Interaction Features ---
    'is_favorite',                 # If the customer has favorited this vendor
    'LOCATION_TYPE',               # e.g., 'Home', 'Work'
    'customer_lat',
    'customer_lon',
    'vendor_lat',
    'vendor_lon',
]

# --- Create the final training dataframe with only the required columns ---
# Keep all rows, even those with missing values
final_training_df = train_merged[required_columns].reset_index(drop=True)

print("\n--- Training Data Ready ---")
print(f"Final training data has {final_training_df.shape[0]} rows and {final_training_df.shape[1]} columns.")
print("Columns:", final_training_df.columns.tolist())
print("\nSample of the final training data:")
print(final_training_df.head())

# Save the final DataFrame to CSV
final_training_df.to_csv('Train/train_merged.csv', index=False)
print("\nMerged training data saved to Train/train_merged.csv")


Loading data...


/tmp/ipykernel_21665/2751923399.py:5: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  train_orders = pd.read_csv('Train/orders.csv')


Preparing and merging data...

Columns in train_merged:
['order_id', 'customer_id', 'item_count', 'grand_total', 'payment_mode', 'promo_code', 'vendor_discount_amount', 'promo_code_discount_percentage', 'is_favorite', 'is_rated', 'vendor_rating_x', 'driver_rating', 'deliverydistance', 'preparationtime', 'delivery_time', 'order_accepted_time', 'driver_accepted_time', 'ready_for_pickup_time', 'picked_up_time', 'delivered_time', 'delivery_date', 'vendor_id', 'created_at_x', 'LOCATION_NUMBER', 'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR', 'gender', 'dob', 'status', 'verified_x', 'language_x', 'created_at_y', 'updated_at_x', 'id', 'authentication_id', 'vendor_lat', 'vendor_lon', 'vendor_category_en', 'vendor_category_id', 'delivery_charge', 'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2', 'prepration_time', 'commission', 'is_haked_delivering', 'discount_percentage', 'vendor_status', 'verified_y', 'rank', 'language_y', 'vendor_rating_y', 'sunday_from_time1', 'sunday_to_time1', 'sun